# [Competition Insights](https://www.kaggle.com/c/lish-moa/discussion/184005)

## Competition and Data Overview 

- 遺伝子発現と細胞生存率レベルをキャプチャするために設計された細胞ベースの分析評価を通じて収集されたデータが提供されます。
- このデータのパターンに基づいて、薬物の作用機序（MoA）を予測する必要があります。
- これはマルチレーベルのコンテストであり、マルチクラスではありません。 一部の行を複数のターゲットに関連付けることができることを意味します。 （これについての詳細）
- 分析評価とデータ収集
    - 分析評価は、標的分子の存在または機能的活性の検出または測定のために生物学および化学で使用される調査ツールです。
    - このデータの収集に使用される分析評価は新しいため、既存の外部データセットは見つかりません。
    - 遺伝子発現の測定はL1000アッセイに基づいています。 ここでそれがどのように機能するかを読むことができます。
    - 細胞生存率の測定は、PRISMアッセイに基づいています。 ここでそれがどのように機能するかを読むことができます。
- 772の遺伝子発現特徴量があり、それぞれ「g-」で示されます。各遺伝子の特徴量は、1つの特定の遺伝子の発現を表します。そのため、アッセイでは772個の個別の遺伝子が監視されています。
    - このデータは、薬物MoAの予測にどのように役立ちますか？これに答える最良の方法は、例を使用することです。下の画像を見ると、cAMPシグナル伝達経路を示す図が表示されます。この図は、2つの異なる小分子の存在（および/または不在）に対する遺伝子発現の依存性を示しています。 PGE_2がその標的受容体に結合すると、シグナル伝達カスケードが活性化され、特定の遺伝子（または遺伝子セット）の発現が増加します。一方、LTB_4がそのターゲットに結合すると、経路を阻害し、遺伝子発現の低下につながります。したがって、既知の遺伝子の発現の増加を観察することにより、薬物が第1のメカニズムを介して作用していると推定することができる。同様に、減少が観察された場合、その薬物は2番目のメカニズムを介して作用していると考えられます。
    - 遺伝子（および細胞株）の実際の名前は、手作りの解決策を防ぐために私たちから隠されていることに注意してください。

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F846065%2F092812f643e57083772497a29399fd8f%2Ffigure1_id125.jpg?generation=1600473140454238&alt=media")

- 100個の細胞生存率機能があり、それぞれ「c-」で示されます。 各細胞の特徴は、1つの特定の細胞株の生存率を表します。 そのため、アッセイでは100個の細胞株がモニターされています。
    - ほとんどの細胞生存率機能の間には非常に強い相関があります。 ここでは、このパターンの考えられる説明を1つ提供します。
    - このデータは、薬物MoAの予測にどのように役立ちますか？ 繰り返しになりますが、これらの機能がターゲットにどのように関連しているかを例を通して説明します。 抗生物質を例にとろう。 ほとんどの抗生物質は、他の種類の細胞を変化させずに、細菌細胞の生存率を大幅に低下させます。
- 元のデータは、提供される前に、変位値正規化を使用して正規化されました。 そのため、これらの機能は正規分布に非常によく適合しています。 私はここで、正規化がどのように機能するか、および生物学的アッセイからのデータのコンテキストで説明する素晴らしい仕事をするビデオを見ました。
- データは-10および+10でクリップされているように見えるため、これらの値でスパイクが発生する可能性があります
- アッセイから収集したデータに加えて、実験の種類、投与量、測定時間の詳細を示す3つの列も提供されます。
    - 治療/対照-cp_vehicle列は、実験が治療（薬物を含む）であるか、対照（薬物を含まない）であるかを示します。 一般的な制御手段はDMSOであり、生物学的影響はごくわずかです。 制御データをどのように使用するかは私たち次第です。 トレーニング中にネガティブサンプルとして使用することも、治療機能を正常化するために利用することもできます。
    - 投与量-cp_dose列は、実験で使用された投与量レベルを示します。 一般的に、より高い用量はより強い効果をもたらします。
    - タイミング-cp_time列は、薬物を追加してから測定が行われるまでの経過時間を示します。 これがデータに与える影響は、メカニズムによって異なります。 たとえば、一部の薬物には不可逆的な効果があり、投薬の72時間後にも確認できます。 他の薬はその時点までにすべての効果を失っており、コントロールと区別がつかないかもしれません。
- 指標はログ損失（別名クロスエントロピー）です。 損失と競争の測定基準の間にギャップがないので、これは有益です。
- 一方、この測定基準は、直感的でなく、このような非常に不均衡な問題には適さないと批判されてきました。 その他のマルチラベル分類指標には次のものがあります。
    - AUC
    - Hamming Loss
    - Jaccard Index
    - Micro/Macro-averaging
    - Exact Match Ratio
    - Top-k Error
- 相互検証は、MultilabelStratifiedKFoldを使用して実行できます。
- 予測しなければならないターゲットメカニズムは206あります。 スコアなしのcsvには、ソリューションに組み込むための追加の402メカニズムが含まれています。
- トレーニングセットのクラスは非常に不均衡です。
    - nfkb_inhibitorは、832エントリを持つ最大の単一クラスを表します。
    - atp-sensitive_potassium_channel_antagonistとerbb2_inhibitorは、それぞれ1つのエントリのみを持つ最小クラスを表します。
- トレーニングデータセットには、ラベルなしの行が最大40％含まれています（スコア付きラベルとスコアなしラベルを考慮すると、最大23％）
- データセット全体には、約5000のユニークな薬物が含まれています。
- 各薬物は、各タイミングおよび投与量レベル（2 * 3）で複数回プロファイルされたため、各薬物には少なくとも6つのエントリが必要です。
- テストデータセット：
    - これはカーネルの競争ですが、公開テストセット全体が提供されます。 つまり、テスト機能をローカルで予測し、submission.csvをアップロードできます。
    - プライベートテストセットはまだ非表示になっていることに注意してください。 プライベートテストセットでスコアを付けるには、推論カーネルを作成し、そこで予測を実行する必要があります。
    - プライベートテストセットは、パブリックテストのサイズの約4倍です。

- 遺伝子発現データ（g-）
- 細胞生存率データ（c-）
- Cpタイプ（化合物（cp_vehicle）または制御摂動（ctrl_vehicle）で処理されたサンプルを示します）
- 治療の期間と用量
- 予測用のMoAラベル